## Count Vector, TFIDF Representations of Text

Working with text generally involves converting it into a format that our model is able to understand, which is mostly numbers. In this notebook, you will have a closer look on two of the most basic and ubiquitiously used formats: 

 - Count Vector
 - TFIDF

You will also build a Machine Learning model on a real world dataset of **BBC News** and perform text classification utilizing the above two formats.

#### Table of Contents
1. About the Dataset
2. Preprocessing Text
3. Working with Count Vector
4. Using TFIDF to improve Count Vector
5. Conclusion
6. Challenge

### 1. About the Dataset

The dataset that you are going to use is a collection of news articles from BBC across 5 major categories, namely:
 
 - Business
 - Entertainment
 - Politics
 - Sport
 - Tech

There are a total of 2225 articles in the dataset, which is a mix of all of the above categories. Let's load the dataset using pandas and have a quick look at some of the articles. 

**Note:** You can get the dataset [here](https://trainings.analyticsvidhya.com/asset-v1:AnalyticsVidhya+LP_DL_2019+2019_T1+type@asset+block@bbc_news_mixed.csv)


In [1]:
import os
from PyPDF2 import PdfReader
import pandas as pd

In [2]:
def pdf_to_text(pdf_path):
    reader = PdfReader(pdf_path)
 
    # printing number of pages in pdf file
    #print(len(reader.pages))
 
    # getting a specific page from the pdf file
    page = reader.pages[0]
 
    # extracting text from page
    text = page.extract_text()
    #print(text)
    return text

In [3]:
df = pd.DataFrame(columns = ['text', 'label'])

In [4]:
mortgage_doc_dir = 'mortgage_documents'
label = 'mortgage'
for filename in os.listdir(mortgage_doc_dir):
    f = os.path.join(mortgage_doc_dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #print(f)
        pdf_text = pdf_to_text(f)
        df = df.append({'text' : pdf_text, 'label' : label}, ignore_index = True)

In [5]:
insurance_doc_dir = 'insurance_documents'
label = 'insurance'
for filename in os.listdir(insurance_doc_dir):
    f = os.path.join(insurance_doc_dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #print(f)
        pdf_text = pdf_to_text(f)
        df = df.append({'text' : pdf_text, 'label' : label}, ignore_index = True)

In [27]:
df.head(12)

,text,label
0,1 \nDEED OF SIMPLE MORTGAGE \n \nTHIS DEED O...,mortgage
1,\n \nFORM NO. 3 \n \n \nForm of Mortgage Dee...,mortgage
2,LD/2239 \n \n \nMEMORANDUM OF DEPOSIT OF TITL...,mortgage
3,Citibank \nMortgage Loan Agreement\n(Applicab...,mortgage
4,MORTGAGE Of Agricultural \nLand With Possessio...,mortgage
5,MORTG...,mortgage
6,\n \n 1 \nClaim Fo...,insurance
7,1a. INSURED’S I.D. NUMBER (FOR PROGR...,insurance
8,DIPLOMA IN INSURANCE SERVICES\nMODULE - 3\nNot...,insurance
9,\n 1 \nIFFCO-TOKIO GENERAL INSURANCE COMPANY...,insurance


In [1]:
import pandas as pd

# Load the dataset
bbc_news = pd.read_csv('bbc_news_mixed.csv')
bbc_news.head()

,text,label
0,Cairn shares slump on oil setback\n\nShares in...,business
1,Egypt to sell off state-owned bank\n\nThe Egyp...,business
2,Cairn shares up on new oil find\n\nShares in C...,business
3,Low-cost airlines hit Eurotunnel\n\nChannel Tu...,business
4,"Parmalat to return to stockmarket\n\nParmalat,...",business


In [6]:
# print first 2 articles
for art in df.text[:2]:
    print(art)

                                         MORTGAGE DEED  FORMAT   
 
                       This deed of   SIMPLE MORTGAGE    executed at                                                                                 
on this the      day of                             201X  by 
Son of                                                          Hindu,  aged about        years and residing    at  
                                                      
Herein  after called the MORTGAGOR  
 
 
 
To and In favor  of 
 
 
 
Thiru.                                           Son of            hindu aged about     years and  residing at 
No.  
 
herein referred to as the MORTGAGEE.  
 
Whereas, the term Mortgagor and Mortgagee, unless repugnant to the context shall mean 
and include their representa tives heirs, successors, executors, administrators, trustees, 
legal representatives and assigns.  
 
Whereas, the Mortgagor herein, is the sole and absolute owner of immovable property 
morefully desc

Now that you have an idea of how your data looks like, let's see the count of each category in the dataset!

In [30]:
# category-wise count
df.label.value_counts()

mortgage     6
insurance    6
Name: label, dtype: int64

### 2. Preprocessing Text

You would have noticed that the labels are in text format, in order to build a model on this dataset you will have to create a mapping between the labels and numbers like 0,1,2,3 this process is called Label Encoding. You can easily label encode your text data using sklearn's [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). Let's have a look at how to do that!

In [7]:
from sklearn.preprocessing import LabelEncoder

# initialize LabelEncoder
lencod = LabelEncoder()
# fit_transform() converts the text to numbers
df.label = lencod.fit_transform(df.label)
# label-wise count
df.label.value_counts()

1    6
0    6
Name: label, dtype: int64

**Note** You'd have noticed in the output of the above code that the text labels have been replaced by numbers. We have a mapping like this - 
 - 0 is Business
 - 1 is Entertainment
 - 2 is Politics
 - 3 is Sport
 - 4 is Tech
 
### 3. Working with Count Vector

Sklearn provides an easy way to create count vectors from a piece of text. You can use the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) to do that. Let's see how simple it is!

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

# initialize count vector
cvec = CountVectorizer(stop_words='english')
# create Bag of Words
bow = cvec.fit_transform(df.text)
# shape of Bag of Words
print('shape of BOW:', bow.shape)
# number of words in the vocabulary
print('No. of words in vocabulary:', len(cvec.vocabulary_))

ValueError: Iterable over raw text documents expected, string object received.

In [23]:
df.text

' 1 \nDEED OF SIMPLE MORTGAGE  \n \nTHIS DEED OF SIMPLE MORTGAGE is made at ______________ this _____ \nday of ________ 200__ \n between  \n \n(1) _______________, son/daughter of _____________ aged about ______ years \nresident at ___________________(hereinafter referred to as the ‘Mortgagor which \nexpression shall unless repugnant to the context or meaning thereof, be deemed \nto mean and include his/her legal heirs, executors and administrators) of the one \npart.  \n \nOR (applicable in case of a couple ) \n \n_________________, son/daughter of _________________ aged about _____ years \nand his/her spouse __________________, son/daughter of ________________ \naged about _____ years both residing at ___________ (hereinafter referred to as the ‘Mortgagors’  which expression sh all unless repugnant to the context or \nmeaning thereof, be deemed to mean and include their legal heirs, executors and \nadministrators) of the one part; and  \n \n(2) ____________________ [Housing Finance C

Let's have a closer look at the Bag of Words that you have just generated.

If you explore the above dataframe, you will find that the Bag of Words representation of the text. Notice that the word "called" appears in the first only once hence there is a 1 at it's index. Now that your BOW is created, let's see just how good is it at classifying the articles in a ML model.

You'll be using [MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) model because it works well with sparse features of text.

In [35]:
df.head()

,text,label
0,1 \nDEED OF SIMPLE MORTGAGE \n \nTHIS DEED O...,1
1,\n \nFORM NO. 3 \n \n \nForm of Mortgage Dee...,1
2,LD/2239 \n \n \nMEMORANDUM OF DEPOSIT OF TITL...,1
3,Citibank \nMortgage Loan Agreement\n(Applicab...,1
4,MORTGAGE Of Agricultural \nLand With Possessio...,1


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# creates a ML model based on parameters
def create_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = MultinomialNB()
    model = model.fit(X_train, y_train)
    return model, X_test, y_test

In [10]:
# create BOW based classification model
model_b, X_test_b, y_test_b = create_model(bow, df.label)

Now that the model is created and trained, have a look at the classification accuracy:

In [18]:
from sklearn.metrics import accuracy_score

# check accuracy 
accuracy_score(y_test_b, model_b.predict(X_test_b))

# df['text'][0]

# model_b.predict(X_test_b)
# X_test_b[0]

0.75

That's a pretty good accuracy and now let's see how can you improve it even further!

### 4. Using TFIDF to improve Count Vector

Just like Count Vector, TFIDF can also be very easily implemented in Python using Sklearn. Here's how you will create a TFIDF representation of your text.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize TFIDF
vec = TfidfVectorizer(max_features=4000, stop_words='english')
# create TFIDF
tfidf = vec.fit_transform(bbc_news.text)
# shape of TFIDF
tfidf.shape

(2225, 4000)

**Note** this time you have a smaller number of columns in shape(4000 as compared to 29192 of previous). This is because we have used the paremeter `max_features` which tells Sklearn to only use 4000 most important words from the entire text in the dataset to build the TFIDF representation. Have a look at how it looks like!

In [16]:
# create a dataframe from the TFIDF
#tfidf_df = pd.SparseDataFrame(tfidf, columns=vec.get_feature_names(), index=bbc_news.index, default_fill_value=0)
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf, columns=vec.get_feature_names(), index=bbc_news.index)

# sample some data points
tfidf_df.iloc[:20, 1000:1050]

,death,debate,debt,debts,debut,dec,decade,decades,december,decent,...,dem,demand,demanded,demands,democracy,democrat,democratic,democrats,dems,denied
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.043469,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.077338,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.105987,0.129173,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.099194,0.181343,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.085918,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.032933,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


If you explore the above dataframe, you will find the TFIDF representation of the text. Notice that each word has a numeric value associated with it, with respect to a column(that in turn represents each document), this is the TFIDF score of that word. Now that your TFIDF is created, let's see just how good is it at classifying the articles in a ML model.

In [17]:
# create TFIDF based classification model
model_t, X_test_t, y_test_t = create_model(tfidf, bbc_news.label)

In [18]:
from sklearn.metrics import accuracy_score

# check accuracy
accuracy_score(y_test_t, model_t.predict(X_test_t))

0.9775280898876404

### 5. Conclusion

 - Notice that using TFIDF word presentation, you were able to build a better model by just using 4000 words as oppossed to the 29,192 words of the BOW. 
 - This is where TFIDF's strength lies which gives the intution that rest of the 25,000+ words weren't adding much useful information to the model and would be common among many documents.
 - You can know more about the word vectors, TFIDF and similar text embeddings in [this comprehensive article](https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/).
 - Finally, note that you could have gotten an even better accuracy by doing preprocessing over the text like Normalization, spelling correction and much more.

### 6. Challenge

If you notice the TFIDF dataframe, words like `demand` `demands` and `demanded` are counted separately this is because the data set isn't normalize yet. I encourage you to go ahead and try to do that using concepts learnt in the previous classes.

In [ ]:
# Your code here